In [ ]:
from experiment.api import mlflow as mlflow_api

In [ ]:
mlflow = mlflow_api.MLFlow()
mlflow.clean()

In [ ]:
run_id = ""

In [ ]:
# serve a model
mlflow.serve_model(run_id)

In [ ]:
# def get_prediction_from_server(row, mlflow: mlflow_api.MLFlow):
#     """
#     This functions receives response from the machine learning server
#     """

#     row = row[:-1]
#     data = {"dataframe_split": {"columns": list(X_train.columns), "data": [row]}}
#     response = mlflow.get_predictions(data)

#     return response.json()